# A [Jupyter](https://jupyter.org/) Kernel for [PowerShell](https://github.com/PowerShell/PowerShell)

This kernel is being written in C#, and in the process, I've taken some messaging code from the [iCSharp kernel](https://github.com/zabirauf/icsharp) and made a generic library for .Net with a re-usable core for anyone who needs to create [Jupyter](https://jupyter.org/) kernels in .Net languages -- so feel free to borrow that if you like (it's under the Apache license).

## Install

I am finally doing a preliminary release: you can download from the releases link, unzip it somewhere, and run the `Install.ps1` script. Note that if you run this on Linux or OS X you should expect to see only "PowerShell (Core)" but on Windows you'll see both -- but only the "PowerShell (Full)" will actually work unless you have PowerShell Core installed in your PATH and working.

## Current Status

At this point, I'm only handling two messages:

* KernelInfo request
* Execute request

The PowerShell kernel is _working_, and returning text output _and errors_ as on the console (see examples below).

## Features

Apart from the built-in Jupyter features, I'm going to add some output enhancements so you can hook into widgets, etc. However, there's none of thata yet, except that:

* If you output HTML, it's rendered. I'm currently detecting this in the most simplistic fashion: by testing if the output starts with "<" and ends with ">". That probably needs work, but it's good enough for now.
* When a command outputs objects, you get the text rendering, but the actual objects are also returned as application/json data.

## PowerShell Core *and Full*

In order to get cross-platform support, this kernel is using [PowerShell Core](https://github.com/PowerShell/PowerShell), which means you'll want to have PowerShell 6 Beta 3 installed to try it out. As of today, it also builds for "Full Framework" Windows PowerShell!

To build it yourself --or to run the "PowerShell (Core)" kernel-- you need [dotnet core 2 preview](https://www.microsoft.com/net/core/preview).  You can build it by running `dotnet restore; dotnet build` from the root. If you want to contribute --and want to build it in Visual Studio-- you need [VS 2017 Preview version 15.3](https://www.visualstudio.com/vs/preview/).

In [1]:
$PSVersionTable

Name                           Value                                                                                    
----                           -----                                                                                    
PSVersion                      5.1.16215.1000                                                                           
PSEdition                      Desktop                                                                                  
PSCompatibleVersions           {1.0, 2.0, 3.0, 4.0...}                                                                  
BuildVersion                   10.0.16215.1000                                                                          
CLRVersion                     4.0.30319.42000                                                                          
WSManStackVersion              3.0                                                                                      
PSRemotingProtocolVersion      2

If we ran that same command in the "PowerShell (Core)" engine, you would see a different, result of course. Incidentally, I don't have any idea what the deal with the "GitCommitId" is -- I need to file a bug on the PowerShell repo about that.

In [1]:
$PSVersionTable

Name                           Value                                                                                    
----                           -----                                                                                    
PSVersion                      6.0.0-beta                                                                               
PSEdition                      Core                                                                                     
GitCommitId                    Could not find file 'C:\Users\Joel\.nuget\packages\system.management.automation\6.0.0-...
OS                             Microsoft Windows 10.0.16215                                                             
Platform                       Win32NT                                                                                  
PSCompatibleVersions           {1.0, 2.0, 3.0, 4.0...}                                                                  
PSRemotingProtocolVersion      2

The rest of these examples actually output the same things regardless of whether you run them in PowerShell (Full) or PowerShell (Core). And I think I will probably rename "PowerShell (Full)" after I sleep on it ...

In [2]:
Get-ChildItem

Directory: C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell


Mode                LastWriteTime         Length Name                                                                   
----                -------------         ------ ----                                                                   
d-----        6/25/2017   1:50 AM                .ipynb_checkpoints                                                     
d-----        6/27/2017  11:54 PM                Jupyter                                                                
d-----        6/28/2017   1:03 AM                Output                                                                 
d-----        6/27/2017  11:54 PM                PowerShell-Kernel                                                      
-a----        6/16/2017  11:57 PM           2581 .gitattributes                                                         
-a----        6/28/2017   1:04 AM           4597 .gitignore                               

In [3]:
$Files = Get-ChildItem -File

## No Output.

Obviously there's no output from that command, because it's all collected in the variable.

I put that here was just to demonstrate that variables stick around so you can use them in other commands in the future, so let's look at the size of those files again:

In [4]:
$Files | Measure Length -Sum

Count    : 8
Average  : 
Sum      : 36152
Maximum  : 
Minimum  : 
Property : Length

In [5]:
Set-Location PowerShell-Kernel

Again, any command with no output just has ... no output.

### But it affects the state.

This means you will be able to use this for literate devops...

Oh, and since I mentioned you can do HTML output, here's PowerShell's built-in ConvertTo-Html

In [6]:
Get-ChildItem -file | ConvertTo-Html Mode, LastWriteTime, Name, Length

Mode,LastWriteTime,Name,Length
-a----,6/26/2017 10:42:55 PM,ExecutionResult.cs,1396
-a----,6/27/2017 9:49:05 PM,Kernel.cs,1306
-a----,6/27/2017 11:55:06 PM,PowerShell-Kernel.csproj,2398
-a----,6/27/2017 8:32:31 PM,PowerShellEngine.cs,7293
-a----,6/26/2017 1:54:08 PM,ScalarHelper.cs,2624


In [7]:
"<img src='https://upload.wikimedia.org/wikipedia/commons/2/2f/PowerShell_5.0_icon.png' />"

<img src='https://upload.wikimedia.org/wikipedia/commons/2/2f/PowerShell_5.0_icon.png' />

### I fixed [3](https://github.com/Jaykul/Jupyter-PowerShell/issues/3) so errors show up:

In [8]:
Get-ChildItem nosuchfile

PathNotFound,Microsoft.PowerShell.Commands.GetChildItemCommand: Get-ChildItem : Cannot find path 'C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\PowerShell-Kernel\nosuchfile' because it does not exist.
At line:1 char:1
+ Get-ChildItem nosuchfile
+ ~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : ObjectNotFound: (C:\Users\Joel\P...rnel\nosuchfile:String) [Get-ChildItem], ItemNotFoundException
    + FullyQualifiedErrorId : PathNotFound,Microsoft.PowerShell.Commands.GetChildItemCommand

Get-ChildItem : Cannot find path 'C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\PowerShell-Kernel\nosuchfile' because it does not exist.
At line:1 char:1
+ Get-ChildItem nosuchfile
+ ~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : ObjectNotFound: (C:\Users\Joel\P...rnel\nosuchfile:String) [Get-ChildItem], ItemNotFoundException
    + FullyQualifiedErrorId : PathNotFound,Microsoft.PowerShell.Commands.GetChildItemCommand